In [32]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai

### Read the sampled dataset with amazon inventory data

In [4]:
df_items=pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [5]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[],169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...",[],24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN


In [ ]:

list(df_items["features"].items())[0]

(0,
 ['【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger)',
  '【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 10 lite and more',
  '【Premium Workmanship】Unique increased friction design allows you to easily unplug the cable from your charger,combine 250d bulletproof fiber core to build a cable so durable

In [7]:
list(df_items["images"].items())[0]

(0,
 [{'thumb': 'https://m.media-amazon.com/images/I/51G07yWoOBL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51G07yWoOBL.jpg',
   'variant': 'MAIN',
   'hi_res': 'https://m.media-amazon.com/images/I/611AVJMH+JL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41c+40oKkKL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41c+40oKkKL.jpg',
   'variant': 'PT01',
   'hi_res': 'https://m.media-amazon.com/images/I/61ihhPW7VCL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51y1YnwiUZL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51y1YnwiUZL.jpg',
   'variant': 'PT02',
   'hi_res': 'https://m.media-amazon.com/images/I/61UkcVETKcL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41Nvr++q39L._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41Nvr++q39L.jpg',
   'variant': 'PT03',
   'hi_res': 'https://m.media-amazon.c

### Preprocess title and features

In [8]:
def preprocess_description(row):
    return f"{row['title']} {''.join(row['features'])}"

In [9]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [13]:
df_items["description"]=df_items.apply(preprocess_description, axis=1)
df_items["image"]=df_items.apply(extract_first_large_image, axis=1)
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,first_large_image,image
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51G07yWoOB...,https://m.media-amazon.com/images/I/51G07yWoOB...
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41bOA5-ogW...,https://m.media-amazon.com/images/I/41bOA5-ogW...
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,USB C Docking Station Dual Monitor for MacBook...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN,https://m.media-amazon.com/images/I/416IzmVKiC...,https://m.media-amazon.com/images/I/416IzmVKiC...
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[2023 Upgraded] Telescopes for Adults Astronom...,169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41wO4J3TT0...,https://m.media-amazon.com/images/I/41wO4J3TT0...
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...","Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41mwlYqT5p...,https://m.media-amazon.com/images/I/41mwlYqT5p...


In [11]:
list(df_items["description"].items())[0]

(0,
 "RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB Type C Fast Charging Cord - Nylon Braided USB C Charger Cable for Galaxy A20/A50/S10/S9/S8+/S8, iPad Pro 2018, Sony XZ, HTC 10, OnePlus 5T, Huawei P9 etc. 【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger)【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 10

In [14]:
list(df_items["image"].items())[0]

(0, 'https://m.media-amazon.com/images/I/51G07yWoOBL.jpg')

### Sample 50 items from the dataset

In [15]:
df_sample=df_items.sample(50,random_state=42)
len(df_sample)

50

In [16]:
data_to_embed=df_sample[["description","image","rating_number","price","average_rating","parent_asin"]].to_dict(orient="records")
data_to_embed


[{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again)[Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation[Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil[Native Palm Rejection]- Res

### Define the embedding function

In [21]:
response=openai.embeddings.create(
    input="random text",
    model="text-embedding-3-small"
)
response
len(response.data[0].embedding)

1536

In [22]:
def get_embedding(text,model="text-embedding-3-small"):
    response=openai.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

### Create Qdrant collection

In [25]:
qdrant_client=QdrantClient(url="http://localhost:6333")

In [31]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

### Embed data

#### Test

In [33]:
pointstruct=PointStruct(
    id=0,
    vector=get_embedding("test text"),
    payload={
        "text": "test text",
        "model": "text-embedding-3-small"
    }
)

In [34]:
pointstruct

PointStruct(id=0, vector=[-0.025677789002656937, 0.00506477989256382, 0.04459746181964874, -0.06313661485910416, -0.03208582103252411, -0.03707829862833023, 0.00230597797781229, -0.011895193718373775, 0.03427764028310776, 0.002538098022341728, 0.025160275399684906, 0.010038234293460846, -0.016119014471769333, 0.01163643691688776, 0.034734271466732025, 0.04106619581580162, -0.026484500616788864, -0.00604272773489356, -0.02958957850933075, 0.050655413419008255, 0.020898401737213135, 0.012047403492033482, 0.0054719410836696625, -0.013889141380786896, -0.003715821076184511, -0.037443604320287704, -0.03540399298071861, 0.015548228286206722, 0.06167539954185486, -0.06709406524896622, 0.022770581766963005, -0.04724591225385666, 0.009718594141304493, -0.03716962784528732, -0.0016809666994959116, 0.023744724690914154, 0.023562071844935417, 0.029467811807990074, 0.024581877514719963, -0.03786979243159294, -0.03777846693992615, -0.012739958241581917, 0.023318536579608917, 0.015692828223109245, 0.

#### Amazon Data

In [35]:
pointstructs=[]
for i,data in enumerate(data_to_embed):
    embedding=get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload=data
        )
    )

In [36]:
pointstructs

[PointStruct(id=0, vector=[0.011265029199421406, -0.014567585662007332, 0.008149511180818081, -0.010393966920673847, -0.04753970727324486, -0.0016192144248634577, -0.04057120904326439, 0.04553038626909256, -0.002382062142714858, -0.009838197380304337, -0.03046046942472458, 0.012665142305195332, -0.02849389798939228, 0.08755514770746231, 0.012462073005735874, 0.005124840419739485, -0.03627467900514603, 0.009693910367786884, -0.01853279024362564, 0.012964403256773949, 0.013423982076346874, 0.013701867312192917, 0.033474452793598175, 0.02922067418694496, 0.018286969512701035, 0.020061157643795013, -0.04057120904326439, -0.02627081796526909, -0.019826024770736694, 0.053353916853666306, -0.013969064690172672, -0.02014666050672531, -0.021600212901830673, -0.051387347280979156, -0.04287979006767273, -0.018244218081235886, -0.017399875447154045, 0.014588961377739906, -0.01949469931423664, -0.001468248083256185, 0.02684796415269375, 0.04176824912428856, 0.009202267974615097, 0.00043252529576420

### Write embedded data to qdrant

In [37]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Define a function for data retrieval

In [38]:
def retrieve_data(query,k=5):
    query_embedding=get_embedding(query)
    results=qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )
    return results

### Test Retrieval

In [40]:
retrieve_data("What kind of tablets can I get?",k=10).points

[ScoredPoint(id=16, version=0, score=0.23814851, payload={'description': 'ESR for iPad Air 5th Generation Case 2022 / iPad Air 4th Case 2020, Rugged Protection, Detachable Magnetic Cover, Adjustable Portrait/Landscape Stand with Raised Screen View, 8 Stand Angles, Black Compatibility: only compatible with iPad Air 5th Generation 2022 and iPad Air 4th Generation 2020; not compatible with any other iPad modelsDetachable Magnetic Cover: keep your screen covered, fold it up into a stand for viewing/writing/portrait modes, or remove it for better ergonomics while gamingRugged Protection: rugged shock-resistant polymer provides robust protection against scratches and dropsStable and Adjustable Stand: magnetically locks to 8 different stand positions that stay stable in bed, on the couch, or anywhere elseStand Your Way: raised screen view, portrait mode, and magnetic fridge mounting let you comfortably view and use your iPad in the office, kitchen, and beyondComplete Customer Support: detaile